#### 
1. 根据提供的kaggle JD评论语料进行文本分类训练
https://www.kaggle.com/datasets/dosonleung/jd_comment_with_label
2. 调整模型训练参数，添加tensorboard跟踪，对比bert冻结和不冻结之间的训练差异。
3. 保存模型进行分类预测。



In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dosonleung/jd_comment_with_label")

print("Path to dataset files:", path)

/opt/miniconda3/envs/py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/peiqi/.cache/kagglehub/datasets/dosonleung/jd_comment_with_label/versions/1


In [3]:
import pandas as pd

# 读取 Excel 文件，并指定引擎为 openpyxl
excel_file_path = '/Users/peiqi/code/AiPremiumClass/李思佳/week10/jd_comment_data.xlsx'
df = pd.read_excel(excel_file_path, engine='openpyxl')
print(len(df))


def load_jd_comments(df):
    jd_comments = {'comment':[], 'lable':[]}

    jd_comm = df['评价内容(content)'].tolist()
    jd_lable = df['评分（总分5分）(score)'].tolist()

    for comm, lb in zip(jd_comm, jd_lable):
        if comm == '此用户未填写评价内容':
            continue
        jd_comments['comment'].append(comm)
        # jd_comments['lable'].append(lb)
        #         # 将标签值减 1，使其范围从 1-5 变为 0-4
        jd_comments['lable'].append(lb - 1)
        
    return jd_comments

jd = load_jd_comments(df)




71818


In [ ]:

import random
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import os



def load_jd_comments(df):
    jd_comments = {'comment':[], 'lable':[]}

    jd_comm = df['评价内容(content)'].tolist()
    jd_lable = df['评分（总分5分）(score)'].tolist()
    #取前1000条
    i = 0 
    for comm, lb in zip(jd_comm, jd_lable):
        if i < 1000:
            if comm == '此用户未填写评价内容':
                continue
            jd_comments['comment'].append(comm)
            # jd_comments['lable'].append(lb)
            # 将标签值减 1，使其范围从 1-5 变为 0-4
            jd_comments['lable'].append(lb - 1)
            i += 1
    print(len(jd_comments['comment']))    
    return jd_comments


def build_collate(tokenizer):
    def collate_fn(batch):
        # 文本分类语料：输入语句，类别标签
        sentents,labels = zip(*batch)
    
        # tokenizer转换
        model_inputs = tokenizer(sentents, return_tensors='pt', padding = True,  truncation = True)
        labels = torch.tensor(labels)

        return model_inputs, labels
    return collate_fn

def train_test_split(X,y,split_rate=0.2):
    # 数据拆分流程
    # 1. 拆分比率
    # 2. 样本随机性
    # 3. 构建拆分索引
    # 4. 借助slice拆分
    split_size = int(len(X) * (1 -split_rate))

    split_index = list(range(len(X)))
    random.shuffle(split_index)
    x_train = [X[i] for i in split_index[:split_size]]
    y_train = [y[i] for i in split_index[:split_size]]

    x_test = [X[i] for i in split_index[split_size:]]
    y_test = [y[i] for i in split_index[split_size:]]

    return (x_train,y_train),(x_test, y_test)

def train_and_eva(train_dl, test_dl, num_epochs, freeze_bert=False, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), model_save_dir="/Users/peiqi/code/AiPremiumClass/李思佳/week10"):
    # 确保保存目录存在
    os.makedirs(model_save_dir, exist_ok=True)
    # 初始化 SummaryWriter
    writer = SummaryWriter('runs/text_classification_experiment')

    best_accuracy = 0.0


    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=5)
    model.to(device)


    if freeze_bert:
        for param in model.bert.parameters():
            param.requires_grad = False
    else:
        for param in model.bert.parameters():
            param.requires_grad = True

    #创建损失函数
    criterion = nn.CrossEntropyLoss()
    #模型优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        print(epoch)
        model.train()
        model.to(device)

        running_loss = 0.0
        for i, (inputs, labels )in enumerate(train_dl):
            print(i)
            # 前向传播
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            if i % 100 == 0:
                avg_loss = running_loss / 100 if i > 0 else running_loss
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i}], Loss: {loss.item():.4f}')
                writer.add_scalar('Training Loss', avg_loss, epoch * len(train_dl) + i)
                running_loss = 0.0

    # 模型评估
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_dl:
            for key in inputs.keys():
                inputs[key] = inputs[key].to(device)
            labels = labels.to(device)

            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    # 计算评估指标
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.4f}, Test F1: {f1:.4f}')
    writer.add_scalar('Test Accuracy', accuracy, epoch)
    writer.add_scalar('Test F1', f1, epoch)

    # 保存最佳模型
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        model_save_path = os.path.join(model_save_dir, f"best_model_epoch_{epoch + 1}.pt")
        torch.save(model.state_dict(), model_save_path)
        print(f"Best model saved at {model_save_path} with accuracy {accuracy:.4f}")

    # 关闭 SummaryWriter
    writer.close()
    return model


if __name__ == '__main__':
    excel_file_path = '/Users/peiqi/code/AiPremiumClass/李思佳/week10/jd_comment_data.xlsx'
    df = pd.read_excel(excel_file_path, engine='openpyxl')
    jd = load_jd_comments(df)

    
    BATCH_SIZE = 128
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    (X_train, y_train),(X_test, y_test) = train_test_split(jd['comment'],jd['lable'])
    
    # 自定义Dataset处理文本数据转换
    train_ds = list(zip(X_train, y_train))
    test_ds = list(zip(X_test, y_test))

    
    # 加载词典创建分词器
    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

    # DataLoader
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=build_collate(tokenizer))
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=build_collate(tokenizer))

    #模型训练
    num_epochs = 2

    # 训练
    m1 = train_and_eva(train_dl, test_dl, num_epochs, freeze_bert=True, device=device) 
    m2 = train_and_eva(train_dl, test_dl, num_epochs, freeze_bert=False, device=device)
